
# AI Assignment 8

## Task 1

1. Download the iris dataset using the link
https://www.kaggle.com/datasets/uciml/iris?select=Iris.csv
2. Construct a neural network with multiple hidden layers
3. Split the dataset into Training (70%) and Testing (30%)
4. Train the data using the constructed neural network with the
following settings:

(i) Loss function= cross entropy, optimizer= Adam, Activation in
output layer= softmax, Activation in hidden layer= sigmoid

(ii) Loss function= cross entropy, optimizer= Adam, Activation in
output layer= softmax, Activation in hidden layer= Relu

![IMG](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Machine+Learning+R/iris-machinelearning.png)

In [43]:
! pip install -Uqq torch sklearn fastai

     |████████████████████████████████| 197 kB 5.3 MB/s 
     |████████████████████████████████| 60 kB 7.1 MB/s 


In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from torch.optim import Adam
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torch.autograd import Variable


le = preprocessing.LabelEncoder()

In [3]:
torch.manual_seed(2)

**Step 1: Download the dataset**

In [4]:
iris = pd.read_csv("https://raw.githubusercontent.com/kurianbenoy/DataminingLab/master/Iris.csv", low_memory=False)
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
iris.describe().T

,count,mean,std,min,25%,50%,75%,max
Id,150.0,75.500000,43.445368,1.0,38.25,75.50,112.75,150.0
SepalLengthCm,150.0,5.843333,0.828066,4.3,5.10,5.80,6.40,7.9
SepalWidthCm,150.0,3.054000,0.433594,2.0,2.80,3.00,3.30,4.4
PetalLengthCm,150.0,3.758667,1.764420,1.0,1.60,4.35,5.10,6.9
PetalWidthCm,150.0,1.198667,0.763161,0.1,0.30,1.30,1.80,2.5


In [6]:
df = iris.drop("Id", axis=1)

In [7]:
df['Species'] = le.fit_transform(df['Species'])

In [8]:
X = df.iloc[:, :-1].values
y = df["Species"].values

**Split the dataset into Training (70%) and Testing (30%)**

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((105, 4), (45, 4), (105,), (45,))

In [10]:
X_Train = X_train.reshape(-1, X_train.shape[1]).astype('float32')
y_train = y_train

X_Test = X_test.reshape(-1, X_test.shape[1]).astype('float32')
y_test = y_test

In [11]:
class IrisDataset(Dataset):
  def __init__(self,X_train, y_train):
    self.X_train = torch.from_numpy(X_train)
    self.y_train = torch.from_numpy(y_train)
    self.length = self.X_train.shape[0]

  def __len__(self):
    return len(self.y_train)

  def __getitem__(self, index):
      return self.X_train[index], self.y_train[index]


In [12]:
training_data = IrisDataset(X_Train, y_train)
validation_data = IrisDataset(X_Test, y_test)

train_dataloader = DataLoader(training_data, batch_size=64)
valid_dataloader = DataLoader(validation_data, batch_size=64, shuffle=True)

In [13]:
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([64, 4])
Labels batch shape: torch.Size([64])


**Network defenition**

In [14]:
class Network1(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Network1, self).__init__()
        self.layer1 = nn.Linear(input_dim, 100)
        self.layer2 = nn.Linear(100, 10)
        self.layer3 = nn.Linear(10, output_dim)
        
    def forward(self, x):
        x = torch.sigmoid(self.layer1(x))
        x = torch.sigmoid(self.layer2(x))
        x = F.softmax(self.layer3(x), dim=1)
        return x


In [15]:
input_dim=4
output_dim=3 

In [16]:
model = Network1(input_dim, output_dim)
model = model.float()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [17]:
print('W:',list(model.parameters())[0].size())
print('b',list(model.parameters())[1].size())

W: torch.Size([100, 4])
b torch.Size([100])


In [18]:
optimizer = Adam(params=model.parameters(), lr=0.01)
criterion=nn.CrossEntropyLoss()

**Training Network**

In [19]:
n_epochs=1000
loss_list=[]

for epoch in range(n_epochs):
    for x, y in train_dataloader:
      optimizer.zero_grad()
      z = model(x.float())
      loss = criterion(z,y)
      loss.backward()
      optimizer.step()
      loss_list.append(loss.data)
      
      
      print(f'epoch {epoch}, loss {loss.item()}')

epoch 0, loss 1.1034886837005615
epoch 0, loss 1.0898600816726685
epoch 1, loss 1.0954985618591309
epoch 1, loss 1.078492283821106
epoch 2, loss 1.0904475450515747
epoch 2, loss 1.0667822360992432
epoch 3, loss 1.0842152833938599
epoch 3, loss 1.0553269386291504
epoch 4, loss 1.0753072500228882
epoch 4, loss 1.0435537099838257
epoch 5, loss 1.0612438917160034
epoch 5, loss 1.0299952030181885
epoch 6, loss 1.0407090187072754
epoch 6, loss 1.01396644115448
epoch 7, loss 1.0151984691619873
epoch 7, loss 0.9958493113517761
epoch 8, loss 0.9878761768341064
epoch 8, loss 0.9757913947105408
epoch 9, loss 0.9603935480117798
epoch 9, loss 0.9538383483886719
epoch 10, loss 0.933303713798523
epoch 10, loss 0.9311452507972717
epoch 11, loss 0.9074084162712097
epoch 11, loss 0.9087414741516113
epoch 12, loss 0.8830872774124146
epoch 12, loss 0.8865369558334351
epoch 13, loss 0.8604621887207031
epoch 13, loss 0.8645079731941223
epoch 14, loss 0.8399372100830078
epoch 14, loss 0.8434367179870605
epoc

**Validating Model**

In [20]:
X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test)

In [21]:
z = model(X_test.float())
yhat = torch.max(z.data,1)

In [22]:
train_acc = torch.sum(y_test == yhat[1])/len(y_test)
print(f"Training accuracy: {train_acc*100}%")

Training accuracy: 100.0%


In [23]:
class Network2(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Network2, self).__init__()
        self.layer1 = nn.Linear(input_dim, 100)
        self.layer2 = nn.Linear(100, 10)
        self.layer3 = nn.Linear(10, output_dim)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x), dim=1)
        return x


In [24]:
model = Network2(input_dim, output_dim)
model = model.float()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print('W:',list(model.parameters())[0].size())
print('b',list(model.parameters())[1].size())

optimizer = Adam(params=model.parameters(), lr=0.01)
criterion=nn.CrossEntropyLoss()

n_epochs=1000
loss_list=[]

for epoch in range(n_epochs):
    for x, y in train_dataloader:
      optimizer.zero_grad()
      z = model(x.float())
      loss = criterion(z,y)
      loss.backward()
      optimizer.step()
      loss_list.append(loss.data)
      
      
      print(f'epoch {epoch+1}, loss {loss.item()}')

W: torch.Size([100, 4])
b torch.Size([100])
epoch 1, loss 1.0981801748275757
epoch 1, loss 1.0452601909637451
epoch 2, loss 1.0696628093719482
epoch 2, loss 1.015746831893921
epoch 3, loss 1.0166972875595093
epoch 3, loss 0.9869368672370911
epoch 4, loss 0.9724121689796448
epoch 4, loss 0.9617031216621399
epoch 5, loss 0.9238921999931335
epoch 5, loss 0.9173368811607361
epoch 6, loss 0.8700936436653137
epoch 6, loss 0.8701635003089905
epoch 7, loss 0.8230085968971252
epoch 7, loss 0.8278483748435974
epoch 8, loss 0.7750490307807922
epoch 8, loss 0.7874675989151001
epoch 9, loss 0.740156888961792
epoch 9, loss 0.7503770589828491
epoch 10, loss 0.7100311517715454
epoch 10, loss 0.7154989838600159
epoch 11, loss 0.6854674816131592
epoch 11, loss 0.6865494847297668
epoch 12, loss 0.6706752181053162
epoch 12, loss 0.6559082269668579
epoch 13, loss 0.6469582915306091
epoch 13, loss 0.6352857351303101
epoch 14, loss 0.6480987071990967
epoch 14, loss 0.6151679158210754
epoch 15, loss 0.6264746

In [25]:
z = model(X_test.float())
yhat = torch.max(z.data,1)

train_acc = torch.sum(y_test == yhat[1])/len(y_test)
print(f"Training accuracy: {train_acc*100}%")

Training accuracy: 97.77777862548828%


## References

1. https://janakiev.com/blog/pytorch-iris/
2. Deep Learning For Coders with fastai & Pytorch - Chapter 4

## Task 2

Construct a medical diagnosis agent to identify the stages of Cirrhosis
Disease.

(i) Download the dataset from [Kaggle prediction dataset](https://www.kaggle.com/datasets/fedesoriano/cirrhosis-prediction-dataset)

(ii) Construct a neural network using multiple hidden layers

(iii) Convert the categorical attributes to numerical and drop the
unnecessary attributes.

(iv) Fill the missing values if any or drop the entire record.

(v) Split the dataset into Training (70%) and Testing (30%)

(vi) Train the data using the constructed neural network with the
following settings: Loss function= cross entropy, optimizer= Adam,
Activation in output layer= softmax, Activation in hidden layer= Relu

(vii)Compute the accuracy of the model.

In [26]:
med = pd.read_csv("https://raw.githubusercontent.com/kurianbenoy/DataminingLab/master/cirrhosis.csv", low_memory=False, index_col='ID')
med.head()

,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
ID,,,,,,,,,,,,,,,,,,,
1,400,D,D-penicillamine,21464,F,Y,Y,Y,Y,14.5,261.0,2.60,156.0,1718.0,137.95,172.0,190.0,12.2,4.0
2,4500,C,D-penicillamine,20617,F,N,Y,Y,N,1.1,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,10.6,3.0
3,1012,D,D-penicillamine,25594,M,N,N,N,S,1.4,176.0,3.48,210.0,516.0,96.10,55.0,151.0,12.0,4.0
4,1925,D,D-penicillamine,19994,F,N,Y,Y,S,1.8,244.0,2.54,64.0,6121.8,60.63,92.0,183.0,10.3,4.0
5,1504,CL,Placebo,13918,F,N,Y,Y,N,3.4,279.0,3.53,143.0,671.0,113.15,72.0,136.0,10.9,3.0


In [27]:
med.describe().T

,count,mean,std,min,25%,50%,75%,max
N_Days,418.0,1917.782297,1104.672992,41.00,1092.7500,1730.00,2613.50,4795.00
Age,418.0,18533.351675,3815.845055,9598.00,15644.5000,18628.00,21272.50,28650.00
Bilirubin,418.0,3.220813,4.407506,0.30,0.8000,1.40,3.40,28.00
Cholesterol,284.0,369.510563,231.944545,120.00,249.5000,309.50,400.00,1775.00
Albumin,418.0,3.497440,0.424972,1.96,3.2425,3.53,3.77,4.64
Copper,310.0,97.648387,85.613920,4.00,41.2500,73.00,123.00,588.00
Alk_Phos,312.0,1982.655769,2140.388824,289.00,871.5000,1259.00,1980.00,13862.40
SGOT,312.0,122.556346,56.699525,26.35,80.6000,114.70,151.90,457.25
Tryglicerides,282.0,124.702128,65.148639,33.00,84.2500,108.00,151.00,598.00
Platelets,407.0,257.024570,98.325585,62.00,188.5000,251.00,318.00,721.00


In [28]:
def advanced_describe(df):
    # get descriptive stats for dataframe for 'all' column dtypes
    desc = df.describe(include='all').T
    desc.drop(['top', 'freq', 'unique'], axis=1, inplace=True)
    
    # update column counts (df.describe() returns NaN for non-numeric cols)
    counts = pd.Series({ col: df[col].count() for col in df.columns })
    desc.update(counts.to_frame('count'))
    
    # add missing count/%
    missings = df.isnull().sum()
    desc = pd.concat([desc, missings.to_frame('missing')], axis=1)
    desc['missing%'] = (desc['missing'] / len(desc)).round(2)

    # add unique counts/%
    uniques = pd.Series({ col: len(df[col].unique()) for col in df.columns })
    desc = pd.concat([desc, uniques.to_frame('unique')], axis=1)
    desc['unique%'] = (desc['unique'] / len(desc)).round(2)
    
    unique_vals = pd.Series({ col: df[col].unique() for col in df.columns if len(df[col].unique()) < 20 })
    desc = pd.concat([desc, unique_vals.to_frame('unique_values')], axis=1)
    
    # add col dtype
    dtypes = pd.Series({ col: df[col].dtype for col in df.columns })
    desc = pd.concat([desc, dtypes.to_frame('dtype')], axis=1)
    
    return desc


In [29]:
advanced_describe(med)

,count,mean,std,min,25%,50%,75%,max,missing,missing%,unique,unique%,unique_values,dtype
N_Days,418,1917.782297,1104.672992,41.0,1092.75,1730.0,2613.5,4795.0,0,0.00,399,21.00,NaN,int64
Status,418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.00,3,0.16,"[D, C, CL]",object
Drug,312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106,5.58,3,0.16,"[D-penicillamine, Placebo, nan]",object
Age,418,18533.351675,3815.845055,9598.0,15644.5,18628.0,21272.5,28650.0,0,0.00,344,18.11,NaN,int64
Sex,418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.00,2,0.11,"[F, M]",object
Ascites,312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106,5.58,3,0.16,"[Y, N, nan]",object
Hepatomegaly,312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106,5.58,3,0.16,"[Y, N, nan]",object
Spiders,312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106,5.58,3,0.16,"[Y, N, nan]",object
Edema,418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.00,3,0.16,"[Y, N, S]",object
Bilirubin,418,3.220813,4.407506,0.3,0.8,1.4,3.4,28.0,0,0.00,98,5.16,NaN,float64


In [30]:
med.shape

(418, 19)

**Dropping nan values**

In [31]:
med.isna().sum()

N_Days             0
Status             0
Drug             106
Age                0
Sex                0
Ascites          106
Hepatomegaly     106
Spiders          106
Edema              0
Bilirubin          0
Cholesterol      134
Albumin            0
Copper           108
Alk_Phos         106
SGOT             106
Tryglicerides    136
Platelets         11
Prothrombin        2
Stage              6
dtype: int64

In [32]:
# From solution: https://stackoverflow.com/questions/39128856/python-drop-row-if-two-columns-are-nan
med.dropna(subset=['Drug','Ascites', 'Hepatomegaly', 'Spiders','Cholesterol','Platelets','Tryglicerides', 'Copper'], how='any', inplace=True)

In [33]:
med.isna().sum()

N_Days           0
Status           0
Drug             0
Age              0
Sex              0
Ascites          0
Hepatomegaly     0
Spiders          0
Edema            0
Bilirubin        0
Cholesterol      0
Albumin          0
Copper           0
Alk_Phos         0
SGOT             0
Tryglicerides    0
Platelets        0
Prothrombin      0
Stage            0
dtype: int64

In [34]:
# Cleaned dataset
med.shape

(276, 19)

In [35]:
med.head()

,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
ID,,,,,,,,,,,,,,,,,,,
1,400,D,D-penicillamine,21464,F,Y,Y,Y,Y,14.5,261.0,2.60,156.0,1718.0,137.95,172.0,190.0,12.2,4.0
2,4500,C,D-penicillamine,20617,F,N,Y,Y,N,1.1,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,10.6,3.0
3,1012,D,D-penicillamine,25594,M,N,N,N,S,1.4,176.0,3.48,210.0,516.0,96.10,55.0,151.0,12.0,4.0
4,1925,D,D-penicillamine,19994,F,N,Y,Y,S,1.8,244.0,2.54,64.0,6121.8,60.63,92.0,183.0,10.3,4.0
5,1504,CL,Placebo,13918,F,N,Y,Y,N,3.4,279.0,3.53,143.0,671.0,113.15,72.0,136.0,10.9,3.0


**Transforming categorical attributes to numerical attributes with label encoding**

In [36]:
# med['Status'] = le.fit_transform(med['Status'])
# med['Drug'] = le.fit_transform(med['Drug'])
# med['Sex'] = le.fit_transform(med['Sex'])
# med['Ascites'] = le.fit_transform(med['Ascites'])
# med['Hepatomegaly'] = le.fit_transform(med['Hepatomegaly'])
# med['Spiders'] = le.fit_transform(med['Spiders'])
# med['Edema'] = le.fit_transform(med['Edema'])


In [37]:
# med["Bilirubin"] = med["Bilirubin"].astype(np.float32)
# med["Cholesterol"] = med["Cholesterol"].astype(np.float32)
# med["Albumin"] = med["Albumin"].astype(np.float32)
# med["Copper"] = med["Copper"].astype(np.float32)
# med["Alk_Phos"] = med["Alk_Phos"].astype(np.float32)
# med["SGOT"] = med["SGOT"].astype(np.float32)
# med["Tryglicerides"] = med["Tryglicerides"].astype(np.float32)
# med["Platelets"] = med["Platelets"].astype(np.float32)
# med["Prothrombin"] = med["Prothrombin"].astype(np.float32)
# med["Stage"] = med["Stage"].astype(int)

In [38]:
# med["Stage"].value_counts()

In [39]:
# med.head()

**Train/Test Split**

In [55]:
med_X = med.iloc[:, :-1]
med_y = med.iloc[:, -1]
med_X.shape, med_y.shape

((276, 18), (276,))

In [60]:
X_train, X_test, y_train, y_test = train_test_split(med_X, med_y, test_size=0.3, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((193, 18), (83, 18), (193,), (83,))

In [61]:
X_train

,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin
ID,,,,,,,,,,,,,,,,,,
44,3428,D,Placebo,13727,F,N,Y,Y,Y,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,199.0,11.7
295,877,CL,D-penicillamine,12912,M,N,N,N,N,2.4,646.0,3.83,102.0,855.0,127.00,194.0,306.0,10.3
130,1413,D,Placebo,16154,F,N,Y,Y,N,17.4,1775.0,3.43,205.0,2065.0,165.85,97.0,418.0,11.5
67,2769,D,Placebo,18733,F,N,N,N,N,1.1,466.0,3.91,84.0,1787.0,328.60,185.0,261.0,10.0
111,2350,CL,D-penicillamine,15031,F,N,N,N,N,5.5,528.0,4.18,77.0,2404.0,172.05,78.0,467.0,10.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,2050,C,Placebo,20684,F,N,Y,N,N,0.9,360.0,3.65,72.0,3186.0,94.55,154.0,269.0,9.7
82,3574,D,D-penicillamine,24585,F,N,N,N,N,4.5,472.0,4.09,154.0,1580.0,117.80,272.0,412.0,11.1
120,2033,CL,D-penicillamine,12839,M,N,N,N,N,3.5,325.0,3.98,444.0,766.0,130.20,210.0,344.0,10.6


## Training Network with fastai

- It uses internally emedding module, with Adam optimizer and Relu as activation???

In [62]:
from fastai.basics import *
from fastai.tabular.all import *

In [63]:
dep_var = "Stage"
cont, cat = cont_cat_split(med, 1, dep_var=dep_var)
procs=[Categorify, FillMissing, Normalize]

In [64]:
dls = TabularDataLoaders.from_df(med, procs=procs, cat_names=cat, cont_names=cont, 
                                 y_names=dep_var,)

In [65]:
learn = tabular_learner(dls)
learn.fit_one_cycle(100)

epoch,train_loss,valid_loss,time
0,10.324608,10.883710,00:00
1,10.208941,10.910127,00:00
2,10.122201,10.920656,00:00
3,10.095407,10.907411,00:00
4,10.115729,10.884459,00:00
5,10.090880,10.814119,00:00
6,10.053436,10.708447,00:00
7,9.955706,10.571459,00:00
8,9.935120,10.427934,00:00
9,9.878119,10.293011,00:00
